## Phase determination

In [2]:
# imports from packages

#import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
#from matplotlib.gridspec import GridSpec # gridspec for nested subfigures
import matplotlib.dates as mdates
import glob
import os

import data_selection
import create_vars
import numb_conc # functions relating to number concentration calculations


In [3]:
# --- Global formatting/settings/variables

sample_rate = 5 # alternatives: 1 s, 5 s, 12 s.

# formatting for only showing time on the x-axis for plots per flight
# Use by setting axs[1,0].xaxis.set_major_formatter(formatter) for each axis where only time should show
formatter = mdates.DateFormatter('%H:%M')

In [4]:
# --- Data import

main_path = '/home/ninalar/Documents/MC2/Results_2022-islas/Processed/ISLAS_processed' # regular path
file_struct = f'/microphy_{sample_rate}s*.nc' # structure of cip text-file names

# get all the .nc files in the main path
files = glob.glob(main_path + file_struct)

# Exclude the file containing 'IS22-09' (flew over land)
files_to_exclude = [f for f in files if 'IS22-09' in os.path.basename(f)]
files_to_include = [f for f in files if f not in files_to_exclude]


ds = xr.open_mfdataset(files_to_include, combine='by_coords', combine_attrs='drop_conflicts') # drop the IS22-09 flight

In [5]:
# Main selection of data
pre_ds, info = data_selection.preselect_ds(ds) # do preselection with default values (T>-30, lat>70)
incloud_ds, incloud_desc = data_selection.sel_incloud_values(pre_ds) # select incloud values with default method (LWC_TWC_th)

# Get short and long descriptions
short_desc = info[0]+'_'+ incloud_desc[0]
long_desc = info[1]+incloud_desc[1]

In [6]:
# --- Preparations for altitude based plots

# need to add altitude binning information(based on in-cloud values9
incloud_ds, altitude_bins, bin_labels = create_vars.add_alt_bins(incloud_ds) # add altitude binning information

In [13]:
incloud_ds['SV_CIP'].mean(dim='time').values

array([0.00107993, 0.00425003, 0.0094058 , 0.01644274, 0.02525633,
       0.03574206, 0.04779541, 0.0613119 , 0.06601619, 0.06479367,
       0.06357116, 0.06234863, 0.06112611, 0.05990358, 0.05868106,
       0.05745854, 0.05623602, 0.0550135 , 0.05379098, 0.05256845,
       0.05134593, 0.05012341, 0.04890088, 0.04767837, 0.04645584,
       0.04523332, 0.0440108 , 0.04278827, 0.04156575, 0.04034324,
       0.0391207 , 0.03789819, 0.03667566, 0.03545314, 0.03423062,
       0.03300809, 0.03178558, 0.03056305, 0.02934053, 0.02811801,
       0.02689549, 0.02567296, 0.02445044, 0.02322792, 0.0220054 ,
       0.02078287, 0.01956035, 0.01833783, 0.01711531, 0.01589279,
       0.01467026, 0.01344774, 0.01222522, 0.0110027 , 0.00978018,
       0.00855765, 0.00733513, 0.00611261, 0.00489009, 0.00366757,
       0.00244504, 0.00122252, 0.        , 0.        ], dtype=float32)

## Checking and testing threshold limits
from the Evans article ++

In [ ]:
# Defining the number concentration threshold limit. The lowest possible number concentration is 1 per sample volume


## Calculating SLF and phase determination

In [7]:
# calculating SLF
incloud_ds['SLF'] = incloud_ds['LWC corr']/(incloud_ds['LWC corr']+incloud_ds['IWC100'])
incloud_ds['LWC corr'].values

array([1.64504930e-05, 0.00000000e+00, 0.00000000e+00, ...,
       1.09485513e-01, 1.38962296e-05, 2.52436811e-05], shape=(5655,))

In [8]:
# Phase determination: mass-based
# categorize cloud phase based on Korolev et. al. 2003

# list of conditions to select categories
SLF_conds = [
    (incloud_ds['SLF'] < 0.1),
    (incloud_ds['SLF'] > 0.9),
    (incloud_ds['SLF'] >= 0.1) & (incloud_ds['SLF'] <= 0.9)
]
# list of categories to return
phase = [
    'ice',
    'liquid',
    'mixed-phase'
]
# set phase
incloud_ds['phase_mass']=np.select(SLF_conds,phase,"ERROR")
incloud_ds['phase_mass']

# count phase
phase_count_mass = incloud_ds['phase_mass'].to_pandas().value_counts()
print(phase_count_mass)

phase_mass
ice            4262
mixed-phase     905
liquid          488
Name: count, dtype: int64


In [9]:
# Phase determination: Concentration-based
# categorize cloud phase based on Evans et. al. 2003
# CDP NT > 2 cm-3
# CIP NT > 0.1 L-1 converted to m-3: multiply by 1000 = 100 m-3

# list of conditions to select categories
SLF_conds = [
    (incloud_ds['NT100'] > 100) & (incloud_ds['Number Conc corr'] < 2),
    (incloud_ds['Number Conc corr'] > 2),
    (incloud_ds['NT100'] > 100) & (incloud_ds['Number Conc corr'] > 2)
]
# list of categories to return
phase = [
    'ice',
    'liquid',
    'mixed-phase'
]
# set phase
incloud_ds['phase_conc']=np.select(SLF_conds,phase,"ERROR")
incloud_ds['phase_conc']

# count phase
phase_count_mass = incloud_ds['phase_conc'].to_pandas().value_counts()
print(phase_count_mass)


phase_conc
ice       4089
liquid    1566
Name: count, dtype: int64


In [28]:
# Checking the NT values
# CDP
print(f'CDP NT units: {incloud_ds['Number Conc corr'].attrs['unit']}')
print(f'max: {incloud_ds['Number Conc corr'].values.max()}')
print(f'min: {incloud_ds['Number Conc corr'].values.min()}')
print(f'All CDP NT: {len(incloud_ds['Number Conc corr'].values)}')
print('--------------------------')



# all liquid values
liq_mask = (incloud_ds['Number Conc corr']>=2).compute()
lq_count = incloud_ds.where(liq_mask, drop = True)
print(f'all liquid: {len(lq_count['Number Conc corr'].values)}')

# only liquid values
only_liq_mask = (lq_count['NT100']<100).compute()
only_lq_count = lq_count.where(only_liq_mask, drop = True)
print(f'only liquid: {len(only_lq_count['Number Conc corr'].values)}')


# all ice:
ice_mask = (incloud_ds['NT100']>=100).compute()
ice_count = incloud_ds.where(ice_mask, drop = True)
print(f'all ice: {len(ice_count['NT100'].values)}')

# only ice:
only_ice_mask = (ice_count['Number Conc corr']<2).compute()
only_ice_count = ice_count.where(only_ice_mask, drop = True)
print(f'only ice: {len(only_ice_count['NT100'].values)}')

#mixed phase
mixed_mask = (ice_count['Number Conc corr']>=2).compute()
mixed_count=ice_count.where(mixed_mask, drop=True)
print(f'mixed: {len(mixed_count['time'].values)}')         

CDP NT units: #/cm^3
max: 603.1792324653054
min: 0.0
All CDP NT: 5655
--------------------------
all liquid: 1566
only liquid: 75
all ice: 5580
only ice: 4089
mixed: 1491


In [22]:
ice_count

<xarray.Dataset> Size: 22MB
Dimensions:                 (time: 5580, Vector64: 64, Vector40: 40,
                             CDP_Bin: 30, phase_mass: 5655, phase_conc: 5655)
Coordinates:
  * time                    (time) datetime64[ns] 45kB 2022-03-22T11:39:01 .....
    lat                     (time) float32 22kB 70.2 70.22 70.22 ... 74.96 74.95
    lon                     (time) float32 22kB 20.38 20.37 ... 20.88 20.88
    alt                     (time) float32 22kB 3.863e+03 3.866e+03 ... 840.7
  * CDP_Bin                 (CDP_Bin) int64 240B 1 2 3 4 5 6 ... 26 27 28 29 30
    islasid                 (time) <U7 156kB 'IS22-02' 'IS22-02' ... 'IS22-11'
    altitude_bin            (time) int64 45kB 3750 3750 3750 ... 450 750 750
  * phase_mass              (phase_mass) <U11 249kB 'ice' 'ice' ... 'ice' 'ice'
  * phase_conc              (phase_conc) <U11 249kB 'ice' 'ice' ... 'ice' 'ice'
Dimensions without coordinates: Vector64, Vector40
Data variables: (12/71)
    base_time               (time) datetime64[ns] 45kB 2022-03-22T11:37:21 .....
    utc_time                (time) float64 45kB dask.array<chunksize=(1885,), meta=np.ndarray>
    TAS                     (time) float32 22kB dask.array<chunksize=(1885,), meta=np.ndarray>
    MIDBINS                 (time, Vector64) float32 1MB dask.array<chunksize=(1885, 64), meta=np.ndarray>
    ACTIVETIME              (time) float32 22kB dask.array<chunksize=(1885,), meta=np.ndarray>
    SA                      (time, Vector64) float32 1MB dask.array<chunksize=(1885, 64), meta=np.ndarray>
    ...                      ...
    TAS correction factor   (time) float64 45kB dask.array<chunksize=(1885,), meta=np.ndarray>
    Number Conc corr        (time) float64 45kB dask.array<chunksize=(1885,), meta=np.ndarray>
    LWC corr                (time) float64 45kB dask.array<chunksize=(1885,), meta=np.ndarray>
    SV                      (time) float32 22kB dask.array<chunksize=(1885,), meta=np.ndarray>
    SV_CDP                  (time) float32 22kB dask.array<chunksize=(1885,), meta=np.ndarray>
    SLF                     (time) float64 45kB dask.array<chunksize=(1885,), meta=np.ndarray>
Attributes:
    date_modified:            2025-05-22
    Joint sample rate (sek):  5.0

In [6]:
#TODO: move this to file/module. the version in create_vars does not work right now

def lat_2band_select(lat_bands, ds):
    """Function to create masks for two latitude bands defined by the latitudes in lat_bands
      Parameters
    ----------
    ds
        xarray dataset with all microphy values
    lat_bands
        array of 3 values: lat_min, lat_mid, lat_max, defining the edges of the two lat bands

    Returns
    -------
    lat_mask_north
        mask to select values in ds that is within the northern latitude band
    lat_mask_south
        mask to select values in ds that is within the southern latitude band
    count_dict
        dictionary with following information:
            - count_south: number of observations in the southern region
            - count_north: number of observations in the northern region
            - lat_bands: array of the latitudes used to define the two regions

    """

    lat_min = lat_bands[0] # unpack lat selection
    lat_mid = lat_bands[1] # unpack lat selection
    lat_max = lat_bands[2] # unpack lat selection

    # count number of values between different latitudes and add to dictionary
    lat_values = ds['lat'].values  # Access the latitude coordinate
    
    count_dict = {'count_south': ((lat_values >= lat_min) & (lat_values <= lat_mid)).sum().item(),
                  'count_north': ((lat_values >= lat_mid) & (lat_values <= lat_max)).sum().item(),
                 'lat_bands': lat_bands}

    #return count_dict
    # Compute the boolean masks for latitude conditions
    lat_mask_north = (ds['lat'].values < lat_max) & (ds['lat'].values >= lat_mid)
    lat_mask_south = (ds['lat'].values < lat_mid) & (ds['lat'].values >= lat_min)

    # Example of filtering a dataset on masks
    #ds_filtered_north = ds.where(lat_mask_north, drop=True)
    #ds_filtered_south = ds.where(lat_mask_south, drop=True)

    #print(f'number of values in 2 bands defined by {lat_bands}:')
    #print(f'count_south: {len(ds_filtered_south.lat)},count_north: {len(ds_filtered_north.lat)}')
    
    return lat_mask_north, lat_mask_south, count_dict


In [7]:
# Mapping variables and set region

# dataset coordinate limits
lon_min = incloud_ds.lon.values.min() 
lon_max = incloud_ds.lon.values.max()
lat_min = incloud_ds.lat.values.min() 
lat_max = incloud_ds.lat.values.max()
lat_mid = lat_min + ((lat_max-lat_min)/2) # separation point for the southern marine and northern marine

# --- Split for southern marine, northern marine and land datasets (in cloud)

# set lat-bands as boundaries for regions (as an array)
lat_bands = [lat_min, lat_mid, lat_max]

# Get the mask to use to select northern and southern band data
lat_mask_north, lat_mask_south, set_val_count_dict = lat_2band_select(lat_bands, ds)

# set the value of region based on the two masks
incloud_ds['region'] = xr.where(lat_mask_north, 'north', xr.where(lat_mask_south, 'south', 'none'))

# drop values where region is 'none'
no_region_mask = (incloud_ds['region']=='none').compute()
incloud_ds = incloud_ds.where(~no_region_mask, drop=True)

In [8]:
# separate Northern and Southern Marine data
# - Northern marine
north_mask = (incloud_ds['region']=='north').compute()
north_ds = incloud_ds.where(north_mask, drop = True)

# - Southern marine
south_ds = incloud_ds.where(incloud_ds['region']=='south', drop = True)
